In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
from src import parse

In [ ]:
def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

In [ ]:
# ASSUMPTIONS
MEAL_CAPACITY = 10000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

In [ ]:
restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200405/restaurants.csv').set_index('code')

In [ ]:
restaurants.shape

In [ ]:
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200405/hospitals.csv'))
schedule = hospitals.iloc[:, 5:]

In [ ]:
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200405/vips.csv'))
vip_schedule = vip_hospitals.iloc[:, 5:]
vip_schedule.shape

In [ ]:
results = sort_schedule(parse.parse_from_results_csv('../../examples/boston/20200405/results.csv'))
results.shape

In [ ]:
results_schedule = results.iloc[:, 5:]
results_schedule_columns = results_schedule.columns

In [ ]:
log = []
vip_log = []
for i, row in results_schedule.iterrows():
    for j, val in enumerate(row):
        if not pd.isnull(val):
            log.append([
                hospitals.iloc[i, 1], 
                hospitals.iloc[i, 4],
                results_schedule_columns[j], 
                val, 
                schedule.iloc[i, j], 
                restaurants[results_schedule_columns[j]][val],
            ])

In [ ]:
meal_log = pd.DataFrame(log, columns=['hospital', 'zone', 'meal', 'restaurant', 'quantity', 'restaurant_capacity'])
meal_log.head(10)

## Rules / Validation Checks:
High-level
- [x] Within budget (# meals < capacity)

Restaurants
- [x] Restaurant has capacity for that order
- [x] Restaurant is delivering to one cluster per meal time
- [x] Restaurant is delivering at least 50 meals for a meal time they are assigned
- [x] Restaurant is not delivering to more than 4 dropoffs per meal time
- [x] Restaurant is within their max days per week

Allocation
- [x] VIP Orders are all fulfilled
- [x] All orders are fully-served (this is pretty much intrinsically assumed, given the format of our data)
- [x] Every HP has > 0 orders

In [ ]:
def test(condition, *print_args):
    if not condition:
        print('TEST FAILURE:', print_args[0])
        for arg in print_args[1:]:
            print(arg)

In [ ]:
# NOTHING in here should be null! This means something isn't matching up with our priors in our spreadsheets!
test(meal_log.isnull().sum().sum() == 0, 'we have nulls in our dense meal log set!')

In [ ]:
# Within budget
test(meal_log['quantity'].sum() < MEAL_CAPACITY, 'we are above meal capacity', meal_log['quantity'].sum(), MEAL_CAPACITY)

In [ ]:
# Restauarant assertions
under_capacity = meal_log[meal_log['restaurant_capacity'] < meal_log['quantity']]
test(under_capacity.empty, 'Some restaurants are assinged a meal that is over their capacity')

In [ ]:
# Restaurant is delivering one cluster per meal time
restaurant_delivery_zones = meal_log.groupby(['meal', 'restaurant'])[['zone']].nunique()
multiple_zones = restaurant_delivery_zones[restaurant_delivery_zones['zone'] > 1]
test(multiple_zones.empty, 'Some restaurants are delivering to more than one zone for a meal!', multiple_zones)

In [ ]:
# Restaurant is delivering at least `MIN_MEALS_PER_RUN` per run
restaurant_quantity_per_meal = meal_log.groupby(['meal', 'restaurant'])[['quantity']].sum()
small_orders_per_meal = restaurant_quantity_per_meal[restaurant_quantity_per_meal['quantity'] < MIN_MEALS_PER_RUN]
test(small_orders_per_meal.empty, 'Some restaurants are delivery a low quantity of meals for an order!', small_orders_per_meal)

In [ ]:
# Restaurant is delivering no more than `MAX_DROPOFFS_PER_RUN` dropoffs per run
dropoffs_per_run = meal_log.groupby(['meal', 'zone', 'restaurant'])[['hospital']].count()
too_many_dropoffs = dropoffs_per_run[dropoffs_per_run['hospital'] > MAX_DROPOFFS_PER_RUN]
test(too_many_dropoffs.empty, 'Some restaurants are delivering to too many dropoff points per run!', too_many_dropoffs)

In [ ]:
# Restaurant is within their max days per week

# Roll up all the meals into days
def column_to_day(series):
    days = set(series.map(lambda m: m.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')))
    return len(days)
    

days_worked = meal_log.groupby(['restaurant']).agg({'meal': [('Days Worked', column_to_day)]}).reset_index()
days_worked.columns = ['restaurant', 'Days Worked']
days_with_capacity = days_worked.merge(restaurants[['max_days_per_week']], left_on='restaurant', right_on='code')
days_over_capacity = days_with_capacity[days_with_capacity['Days Worked'] > days_with_capacity['max_days_per_week']]
test(too_many_dropoffs.empty, 'Some restaurants are working more days than requested this week', days_over_capacity)

In [ ]:
# Ensure all VIP orders are fulfilled
vip_schedule_dense = vip_schedule.dropna(how='all', axis=0).dropna(how='all', axis=1)
vip_schedule_dense
vip_orders = []
missing_meals = []
for i, row in vip_schedule_dense.iterrows():
    for j, val in enumerate(row):
        if not pd.isnull(val):
            order = meal_log[(meal_log['hospital'] == hospitals.iloc[i, 1]) & (meal_log['meal'] == vip_schedule_dense.columns[j])]
            vip_orders.append(order)
            if order.empty:
                missing_meals.append((hospitals.iloc[i, 1], vip_schedule_dense.columns[j]))
                
test(len(missing_meals) == 0, 'Some VIP meal orders have not been fulfilled', missing_meals)

In [ ]:
vip_orders_df = pd.concat(vip_orders).reset_index()

In [ ]:
hospital_meals_rollup = meal_log[['hospital', 'quantity']].groupby('hospital').agg({
    'quantity': [('total_meals_received', 'sum'), ('total_orders_fulfilled', 'count')]
}).sort_values([('quantity', 'total_meals_received')], ascending=False).reset_index()
hospital_meals_rollup.columns = ['hospital', 'total_meals_received', 'total_orders_fulfilled']

In [ ]:
vip_orders_rollup = vip_orders_df.groupby('hospital')[['index']].count().reset_index()
vip_orders_rollup.columns = ['hospital', 'vip_meals_count']
vip_orders_rollup

In [ ]:
hospital_meals_rollup = meal_log[['hospital', 'quantity']].groupby('hospital').agg({
    'quantity': [('total_meals_received', 'sum'), ('total_orders_fulfilled', 'count')],
}).sort_values([('quantity', 'total_meals_received')], ascending=False).reset_index()
hospital_meals_rollup.columns = ['hospital', 'total_meals_received', 'total_orders_fulfilled']

hospital_meals_rollup = hospital_meals_rollup.merge(vip_orders_rollup, on='hospital', how='left').fillna(0)

In [ ]:
# Every HP has > 0 orders
hospitals_no_orders = hospital_meals_rollup[hospital_meals_rollup['total_orders_fulfilled'] == 0]
test(hospitals_no_orders.empty, 'Some Hospitals received no orders!', hospitals_no_orders)

## KPIs:
- [X] Meals scheduled for delivery
- [x] Utilization % (number of meals delivered / capacity)
- [x] Meals per Delivery Run
- [x] Total meals and number of orders for each HP
- [x] Total meals and number of orders for each Restaurant
- [x] Equitable meals served: all HPs are within 1 order of each other, excluding VIP orders

In [ ]:
# Number of meals delivered
meals_scheduled = meal_log['quantity'].sum()
meals_scheduled

In [ ]:
# Meal Utilization %
utilization = meals_scheduled / MEAL_CAPACITY
print(str(utilization * 100) + '%')

In [ ]:
grouped_by_meal_restaurant = meal_log.groupby(['meal', 'restaurant']).agg({
    'quantity': ['sum', 'count']
}).sort_values(by='meal', ascending=False)
# grouped_by_meal_restaurant.head(10)

In [ ]:
# Meals delivered by Run, and Number of Dropoff Points
# CAVEAT: this ASSUMES that a restaurant is delivering to a single cluster for one meal time. Careful!
meals_per_run_agg = grouped_by_meal_restaurant.agg(['median', 'mean', 'min', 'max'])
meals_per_run_agg.columns = ['Meals per Meal Time', 'Dropoff Points']
meals_per_run_agg

In [ ]:
# Total number of meals delivered by restaurant
meal_log.groupby('restaurant')[['quantity']].sum().sort_values(by='quantity', ascending=False)

In [ ]:
# Total meals and orders for each Restaurant
meal_log.groupby('restaurant').agg({
    'quantity': [('meal_sum', 'sum'), ('orders_fulfilled', 'count')]
}).sort_values([('quantity', 'meal_sum')], ascending=False)

In [ ]:
# Total numbers of meals and orders received for each HP
# NB: this variable is defined above, as it's used in a validation test
hospital_meals_rollup

## Hospital Equity Check

In [1]:
# Hospital Equity check
max_row = hospital_meals_rollup[
    hospital_meals_rollup['total_orders_fulfilled'] == hospital_meals_rollup['total_orders_fulfilled'
].max()]
min_row = hospital_meals_rollup[
    hospital_meals_rollup['total_orders_fulfilled'] == hospital_meals_rollup['total_orders_fulfilled'
].min()]

order_range = (max_row['total_orders_fulfilled'].values[0] - min_row['total_orders_fulfilled'].values[0])
print('INCLUDING VIP MEALS')
print('RANGE OF ORDERS FOR HOSPITALS:', order_range)
if order_range > 1:
    print('This range should ideally only be 1, once we remove VIP orders!')
print('Note: this is inclusive of VIP orders. Please verify this discrepancy!')
print('------------------------------')
print(
    'MAXIMUM Order for Hospital:', 
    max_row['total_orders_fulfilled'], 
    'for Hospital(s):', 
)
for _, row in max_row.iterrows():
    print('-', row['hospital'], '\tNumber of VIP orders:', row['vip_meals_count'])
print()
print(
    'MINIMUM Order for Hospital:', 
    min_row['total_orders_fulfilled'], 
    'for Hospital(s):', 
)
for _, row in min_row.iterrows():
    print('-', row['hospital'], '\tNumber of VIP orders:', row['vip_meals_count'])


NameError: name 'hospital_meals_rollup' is not defined

In [ ]:
print('Removing VIP Meals from the equation (VIP meals collapse to a single meal for each hospital):')
(hospital_meals_rollup[
    'total_orders_fulfilled'
] - hospital_meals_rollup[
    'vip_meals_count'
].transform(lambda x: max(0, x-1))).describe()